# LSTM Single Model

## Data Description

Input file: Historical Product Demand.csv

Description: CSV data file containing product demand for encoded product id's

Size of Data: (1048575, 5)

Features: Product_Code, Warehouse, Product_Category, Date, Order_Demand

Period: 2012-01-01 ~ 2017-01-09


---

In [1]:
# DataFrame
import pandas as pd
import numpy as np
import random
from datetime import datetime, date

# Preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

# Save the log
import os


# LSTM
import tensorflow as tf

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MSE

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Metric 
# Metric
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error

%config Completer.use_jedi = False

## Data Explore

In [2]:
# Data Loading
df = pd.read_csv('HPD_Augmented_0416.csv')
# convert the string to the datetype
df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df

,Date,Product_Code,Product_Category,Order_Demand
0,2012-01-05 00:00:00,Product_0025,Category_005,1600.000000
1,2012-01-05 03:00:00,Product_0025,Category_005,1633.403702
2,2012-01-05 06:00:00,Product_0025,Category_005,1628.665789
3,2012-01-05 09:00:00,Product_0025,Category_005,1587.586651
4,2012-01-05 12:00:00,Product_0025,Category_005,1513.949924
...,...,...,...,...
116387,2016-12-26 12:00:00,Product_2004,Category_005,1810.945746
116388,2016-12-26 15:00:00,Product_2004,Category_005,1626.979543
116389,2016-12-26 18:00:00,Product_2004,Category_005,1420.229634
116390,2016-12-26 21:00:00,Product_2004,Category_005,1206.795489


In [3]:
print(df.info())
print('-------------------------')
print("")
print("The Number of unique")
print('-------------------------')
print('Product code:\t', df.Product_Code.nunique())
print('Category:\t', df.Product_Category.nunique())
print('-------------------------')
print("The Product Code:")
print("")
for i, code in enumerate(df['Product_Code'].unique()):
    print(i+1, code)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116392 entries, 0 to 116391
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Date              116392 non-null  datetime64[ns]
 1   Product_Code      116392 non-null  object        
 2   Product_Category  116392 non-null  object        
 3   Order_Demand      116392 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 3.6+ MB
None
-------------------------

The Number of unique
-------------------------
Product code:	 8
Category:	 5
-------------------------
The Product Code:

1 Product_0025
2 Product_0739
3 Product_0901
4 Product_1154
5 Product_1248
6 Product_1295
7 Product_1378
8 Product_2004


---

### Split the train and test set
- Input
     data: dataframe with dates and Demand data
     
- output
    - train:  2012-01-01 ~ 2015-06/30 
    - Valid:  2015-07-01 ~ 2015-12-31
    - test :  2016-01-01 ~ 2017-01-06 
    
     
- time_steps: # of the input time steps 
- for_periods: # of the output time steps 

In [4]:
def ts_train_val_test(product_df, time_steps): 

    ts_train_end = len(product_df[product_df['Date']<'2015-07-01']) # train 데이터 종료 인덱스
    ts_val_end = len(product_df[product_df['Date']<'2016-01-01']) # validation 데이터 종료 인덱스
    ts = product_df.filter(['y']).values # y(수요량) 값
    
    # Minmax로 0~1 사이에 값이 오도록 정규화
    sc = MinMaxScaler() # 객체 생성
    ts_scaled = sc.fit_transform(ts) # 전체 y값 정규화
    
    # Train Data
    ts_train_scaled = ts_scaled[:ts_train_end,:]

    X_train = [] 
    y_train = []
    for i in range(time_steps, ts_train_end): 
        X_train.append(ts_train_scaled[i-time_steps:i,0]) # time steps 만큼 sliding window
        y_train.append(ts_train_scaled[i,0])

    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    # Reshape X_train for LSTM -> (batch_size, time_steps, features)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))

    # Validation Data
    ts_val_scaled = ts_scaled[ts_train_end : ts_val_end, :]

    X_val = []
    y_val = []
    for i in range(time_steps, len(ts_val_scaled)):
        X_val.append(ts_val_scaled[i-time_steps : i, 0])
        y_val.append(ts_val_scaled[i, 0])

    X_val = np.array(X_val)
    y_val = np.array(y_val)

    # Reshape X_val for LSTM -> (batch_size, time_steps, features)
    X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1],1))
    
    # Test Data
    ts_test_scaled = ts_scaled[ts_val_end:,:]

    X_test = []
    y_test = product_df.iloc[ts_val_end+time_steps:,:]
    y_test.loc[:, 'y_norm'] = ts_test_scaled[time_steps:].reshape(-1).copy()

    for i in range(time_steps, len(ts_test_scaled)):
        X_test.append(ts_test_scaled[i-time_steps : i, 0])
    
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
    
    return X_train, y_train, X_val, y_val, X_test, y_test, sc

### LSTM

In [17]:
def LSTM_model(X_train, y_train, X_val, y_val, X_test, sc, epochs=15):
    # LSTM 모델 객체 생성
    my_LSTM_model = Sequential() 
    
    # 첫 번째 LSTM 레이어 구성
    # 활성화 함수는 ReLU를 사용하며, return_sequences=True로 지정하여 다음 LSTM 레이어의 입력으로 사용할 수 있도록 함
    my_LSTM_model.add(LSTM(512, activation='relu',return_sequences=True, input_shape=(X_train.shape[1],1)))
    
    # 두 번째 LSTM 레이어 구성
    # 활성화 함수는 ReLU를 사용하며, return_sequences=False로 지정하여 마지막 LSTM 레이어임을 나타냄
    my_LSTM_model.add(LSTM(256, activation = 'relu',return_sequences=False))
    
    # Fully connected 레이어들 추가
    # 마지막 레이어에서는 출력의 unit 개수를 1로 설정하여 1개의 값을 출력
    my_LSTM_model.add(Dense(128))
    my_LSTM_model.add(Dense(64))
    my_LSTM_model.add(Dense(32))
    my_LSTM_model.add(Dense(1))
    
    # 모델 컴파일
    my_LSTM_model.compile(optimizer = "Adam", # Adam optimizer 사용
                         loss = 'mean_squared_error', # 손실 함수로는 평균 제곱 오차 사용
                          metrics=['mape','mae']) # 성능 지표로는 MAPE와 MAE를 사용
    #조기종료 조건
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # 모델 Fitting
    my_LSTM_model.fit(X_train, # 입력 데이터
                      y_train, # 출력 데이터
                      epochs = epochs, # epoch 수
                      batch_size = 16, # batch size
                      validation_data=(X_val, y_val),
                      callbacks=[early_stopping],# validation에 따른 조기종료
                      verbose = 1) # 학습 상태를 출력
    
    # Test 데이터 예측
    LSTM_prediction = my_LSTM_model.predict(X_test) # 예측값 얻기
    LSTM_prediction_normalized = LSTM_prediction # 예측값을 저장하되, normalize된 값 저장
    LSTM_prediction = sc.inverse_transform(LSTM_prediction) # denormalize된 예측값 저장
    
    # 모델 객체와 예측값 반환
    return my_LSTM_model, LSTM_prediction, LSTM_prediction_normalized, sc

### EEMD Single Model

In [6]:
def LSTM_single(product_df, time_steps, epochs):

    # 학습 데이터와 테스트 데이터 분리
    X_train, y_train, X_val, y_val, X_test, y_test, sc = ts_train_val_test(product_df, time_steps)
    
    # LSTM 모델 학습 및 저장
    my_LSTM_model, LSTM_prediction, LSTM_prediction_normalized = LSTM_model(X_train, y_train, X_val, y_val, X_test, sc, epochs)
    
    # 예측 결과 저장
    y_test.reset_index(drop=True, inplace=True)
    pred_df = pd.DataFrame({'Pred': LSTM_prediction.reshape(-1) ,'Pred_norm': LSTM_prediction_normalized.reshape(-1)})
    res_df = pd.concat([y_test, pred_df], axis=1)
    res_df.set_index('Date', inplace=True)
    res_df = res_df.resample('D').first() # 증강된 데이터가 아닌, Actual값들과 비교
        
    # 모델과 result_df
    return my_LSTM_model, res_df

## Plot the result

In [7]:
def actual_pred_plot(product_code, res_df, metric_df, normalize):
    today = date.today()
    """
    Plot the actual vs predition and save the figure in the given directory
    """
    
    save_path = os.path.join("Result", "Single_LSTM_Result", product_code)
    save_name = f'{product_code}_all_result'
    
    title = f"Pred Actual Plot - {product_code}"
    actual = res_df['y']
    pred = res_df['Pred']
    
    if normalize: 
        title += "(Normalized)"
        actual = res_df['y_norm']
        pred = res_df['Pred_norm']
        save_name += "_normalized"
    # Plot   
    plt.figure(figsize=(16, 8))
    plt.title(title, fontsize=20)
    plt.xlabel("Time", fontsize=14)
    plt.ylabel("Order Demand", fontsize=14)
    plt.plot(actual, label ='Actual', alpha=0.6)
    plt.plot(pred, label='Prediction', alpha=0.8)
    plt.legend(loc="upper right")
        
    # Plot 결과 저장
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    # save the figure
    today_date = f'_{today.month:02d}{today.day:02d}'
    plt.savefig(os.path.join(save_path, save_name+'.png'))
    metric_df.to_csv(os.path.join(save_path, save_name+'.csv'))
        
    plt.close('all') # close all figures to free up memory

## Metrics

In [8]:
# Model Metric
def mase(training_series, testing_series, prediction_series):
    n = training_series.shape[0]
    d = np.abs(np.diff(training_series)).sum() / (n-1)
    
    errors = np.abs(testing_series - prediction_series)
    return errors.mean() / d

# Model Metric
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / (actual+1)))

def nrmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, squared=False)
    target_mean = np.mean(y_true)
    nrmse = mse / target_mean
    return nrmse

In [9]:
def calculate_metrics(product_code, res_df, normalize):
    # 정규화 옵션이 True인 경우 정규화된 데이터 사용, 그렇지 않으면 원래 데이터 사용
    if normalize:
        actual = res_df['y_norm']
        pred = res_df['Pred_norm']
    else:
        actual = res_df['y']
        pred = res_df['Pred']

    # 메트릭 계산
    # MASE = mase(np.array(train_series), np.array(actual), pred) 
    MAPE = mape(actual, pred) 
    RMSE = mean_squared_error(actual, pred)**0.5 
    MAE = mean_absolute_error(actual,pred) 
    NRMSE = nrmse(actual,pred) 
    # RMSLE = mean_squared_log_error(actual, pred)**0.5 

    # 계산된 메트릭을 데이터프레임에 추가
    metric_df = pd.DataFrame({'MAPE':[round(MAPE, 4)],
                           'RMSE':[round(RMSE, 4)],
                           'MAE':[round(MAE, 4)],
                           'NRMSE':[round(NRMSE, 4)]},
                            index= [product_code])

    return metric_df

---

## Check the Result

In [10]:
def make_all_result_df(res_df):
    
    res_df.loc[res_df['Pred']<0, 'Pred']=0 # 음수 예측 값은 0으로 대치
    
    # 날짜(Date) 열은 정규화하지 않으므로 제외
    result_norm = res_df[['Pred', 'y']]
    
    # MinMaxScaler를 이용하여 정규화합니다.
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(result_norm)
    
    # 정규화된 데이터를 데이터 프레임에 반영합니다.
    res_df['Pred_norm'] = normalized_data[:,0]
    res_df['y_norm'] = normalized_data[:,1]
    return res_df

In [75]:
def execute_single_LSTM(product_code, time_steps=30, epochs=20):

    product_code = product_code # 예측하고자 하는 코드 입력
    product_df = df[df['Product_Code']== product_code].reset_index(drop=True)
    product_df = product_df[['Date', 'Order_Demand']]
    product_df.rename(columns={'Order_Demand': 'y'}, inplace=True)

    # LSTM 단일 모델
    model, res_df = LSTM_single(product_df, time_steps, epochs) #dictionary, time_steps, epochs
    res_df = make_all_result_df(res_df)
    
    # 모델 Metric과 Pred_Actual Plot 저장
    metric_df_norm = calculate_metrics(product_code, res_df, True)
    metric_df= calculate_metrics(product_code, res_df, False)
    
    actual_pred_plot(product_code, res_df, metric_df_norm, True)
    actual_pred_plot(product_code, res_df, metric_df, False)
    
    return metric_df

In [12]:
product_code = 'Product_0025' # 예측하고자 하는 코드 입력
product_df = df[df['Product_Code']== product_code].reset_index(drop=True)
product_df = product_df[['Date', 'Order_Demand']]
product_df.rename(columns={'Order_Demand': 'y'}, inplace=True)
# LSTM 단일 모델
model, res_df = LSTM_single(product_df, 1, 1) #dictionary, time_steps, epochs
# res_df = make_all_result_df(res_df)

# 모델 Metric과 Pred_Actual Plot 저장
# metric_df_norm = calculate_metrics(product_code, res_df, True)
# metric_df= calculate_metrics(product_code, res_df, False)

# actual_pred_plot(product_code, res_df, metric_df_norm, True)
# actual_pred_plot(product_code, res_df, metric_df, False)

93/93 [==============================] - 1s 4ms/step


In [16]:
all_res_df = make_all_result_df(res_df)
all_res_df

,y,y_norm,Pred,Pred_norm
Date,,,,
2016-01-01,0.0,0.000000,56.458679,0.000000
2016-01-02,0.0,0.000000,56.458679,0.000000
2016-01-03,0.0,0.000000,139.364914,0.013700
2016-01-04,0.0,0.000000,56.458679,0.000000
2016-01-05,3000.0,0.483871,2917.541992,0.472791
...,...,...,...,...
2017-01-02,0.0,0.000000,63.035332,0.001087
2017-01-03,0.0,0.000000,56.458679,0.000000
2017-01-04,0.0,0.000000,97.992676,0.006863


In [13]:
res_df

,y,y_norm,Pred,Pred_norm
Date,,,,
2016-01-01,0.0,0.000000,56.458679,0.004705
2016-01-02,0.0,0.000000,56.458679,0.004705
2016-01-03,0.0,0.000000,139.364914,0.011614
2016-01-04,0.0,0.000000,56.458679,0.004705
2016-01-05,3000.0,0.250000,2917.541992,0.243128
...,...,...,...,...
2017-01-02,0.0,0.000000,63.035332,0.005253
2017-01-03,0.0,0.000000,56.458679,0.004705
2017-01-04,0.0,0.000000,97.992676,0.008166


---

## Whole Process
    - product_code에 str으로 예측하고자 하는 코드를 입력
    - ['Product_0025', 'Product_0739', 'Product_0901', 'Product_1154',
       'Product_1248', 'Product_1295', 'Product_1378', 'Product_2004']

In [79]:
import time
start_time = time.time()

execute_single_LSTM('Product_2004')

elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60
print("")
print("================= END ======================")
print("총 실행 시간: {:.2f} 분".format(elapsed_time_minutes))

91/91 [==============================] - 1s 3ms/step

================= END ======================
총 실행 시간: 0.25 분
